In [2]:
import pandas as pd
import numpy as np

week1 = pd.read_csv('nfl-big-data-bowl-2021/week1.csv')

In [9]:
play1 = week1[(week1['playId'] == 75) & (week1['team'] == 'away')]

In [13]:
play1['time'] = pd.to_datetime(play1['time'], format='%Y-%m-%dT%H:%M:%S')

/var/folders/6v/8jcf0jcn7_vfm2q2r4tz44gh0000gn/T/ipykernel_62568/470589069.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  play1['time'] = pd.to_datetime(play1['time'], format='%Y-%m-%dT%H:%M:%S')


In [73]:
play1['time_diff'] = play1.groupby('displayName')['time'].diff()

/var/folders/6v/8jcf0jcn7_vfm2q2r4tz44gh0000gn/T/ipykernel_62568/1725331901.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  play1['time_diff'] = play1.groupby('displayName')['time'].diff()


In [74]:
play1['time_diff'][play1['time_diff'].isnull()] = pd.Timedelta(0)

play1['time_acc_sec'] = play1.groupby('displayName')['time_diff'].transform(lambda x: x.map(lambda x: x.microseconds).cumsum()).div(1e6)

/var/folders/6v/8jcf0jcn7_vfm2q2r4tz44gh0000gn/T/ipykernel_62568/977950757.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  play1['time_diff'][play1['time_diff'].isnull()] = pd.Timedelta(0)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:8870: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/var/folders/6v/8jcf0jcn7_vfm2q2r4tz44gh0000gn/T/ipykernel_62568/977950757.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

In [84]:
plays = pd.read_csv('nfl-big-data-bowl-2021/plays.csv')

In [88]:
play1 = play1.merge(plays, on=['playId', 'gameId'])

In [89]:
play1.head()

,time,x,y,s,a,dis,o,dir,event,nflId,...,preSnapHomeScore,gameClock,absoluteYardlineNumber,penaltyCodes,penaltyJerseyNumbers,passResult,offensePlayResult,playResult,epa,isDefensivePI
0,2018-09-07 01:07:14.599000+00:00,91.73,26.67,0.00,0.01,0.02,289.57,240.93,None,310.0,...,0.0,15:00:00,90.0,NaN,NaN,C,10,10,0.261827,False
1,2018-09-07 01:07:14.599000+00:00,91.35,44.16,0.02,0.03,0.01,290.45,16.86,None,2495454.0,...,0.0,15:00:00,90.0,NaN,NaN,C,10,10,0.261827,False
2,2018-09-07 01:07:14.599000+00:00,90.78,36.15,0.00,0.00,0.00,257.61,193.97,None,2533040.0,...,0.0,15:00:00,90.0,NaN,NaN,C,10,10,0.261827,False
3,2018-09-07 01:07:14.599000+00:00,98.25,26.63,0.01,0.01,0.00,265.88,194.51,None,2543583.0,...,0.0,15:00:00,90.0,NaN,NaN,C,10,10,0.261827,False
4,2018-09-07 01:07:14.599000+00:00,91.29,21.83,0.01,0.02,0.01,250.34,151.83,None,2555415.0,...,0.0,15:00:00,90.0,NaN,NaN,C,10,10,0.261827,False


In [146]:
play1['yardline_100'] = play1['absoluteYardlineNumber'].copy()
play1['yardline_first'] = np.where(play1['yardline_100'].gt(play1['x'].max()),
                                  play1['yardline_100'].add(play1['yardsToGo']),
                                  play1['yardline_100'].sub(play1['yardsToGo']))

In [147]:
play1['yardline_first']

0      75.0
1      75.0
2      75.0
3      75.0
4      75.0
       ... 
349    75.0
350    75.0
351    75.0
352    75.0
353    75.0
Name: yardline_first, Length: 354, dtype: float64

In [233]:
import altair as alt

alt.themes.enable('fivethirtyeight')

players = alt.Chart(play1.drop(['time', 'time_diff'], axis=1)).encode(
    x=alt.X('x:Q', title="Yard Line", scale=alt.Scale(zero=True)),
    y=alt.Y('y:Q', axis=None),
    color=alt.Color('displayName:N', title='Player',
                    scale=alt.Scale(scheme='category10'))
).mark_line(width=20)

scrimmage = alt.Chart(play1.drop(['time', 'time_diff'], axis=1)).encode(
    x=alt.X('yardline_100:Q')
).mark_rule(color='blue')

first = alt.Chart(play1.drop(['time', 'time_diff'], axis=1)).encode(
    x=alt.X('yardline_first:Q')
).mark_rule(color='yellow')

In [235]:
((players + scrimmage + first)
 .configure_view(fill='#348C31', fillOpacity=0.7)
 .properties(height=55*5, width=100*5, title=alt.TitleParams('Routes on First Play', anchor='middle')))

alt.LayerChart(...)